# 📊 Proyecto: Análisis y visualización de la calidad del aire: una exploración de contaminantes atmosféricos y su relación con el PM2.5
**Curso:** Samsung Innovation Campus – Módulo de Python (Ecuador 2025)  
**Seccion:** EC03  

---

## 🧩 Módulo 1: Adquisición de Datos

En esta sección se carga el dataset, se explora su estructura general y se validan los rangos de los contaminantes.

El dataset contiene mediciones de diferentes gases y partículas contaminantes en el aire, registradas con fecha y hora.


### 📘 Descripción general de las columnas del dataset

| Columna | Descripción | Relevancia |
|----------|--------------|-------------|
| 📅 **Date** | Fecha y hora del registro | Permite analizar variaciones temporales. |
| 🏭 **CO** | Monóxido de carbono (ppm) | Gas tóxico, indicador de combustión incompleta. |
| 🚗 **NO** | Óxido nítrico | Se genera en la quema de combustibles fósiles. |
| 🚙 **NO2** | Dióxido de nitrógeno | Contaminante urbano, precursor del ozono. |
| 🌫️ **O3** | Ozono troposférico | Se forma con NOx + luz solar; irritante respiratorio. |
| 🌋 **SO2** | Dióxido de azufre | Proviene de la quema de carbón y petróleo. |
| ☁️ **PM2.5** | Partículas finas < 2.5 μm | Altamente dañinas, ingresan a los pulmones. |
| 🌧️ **PM10** | Partículas < 10 μm | Menos dañinas, quedan en vías respiratorias. |
| 🍃 **NH3** | Amoníaco | De origen agrícola, contribuye a partículas secundarias. |


In [430]:
# ===== Importar librerías principales ====
import pandas as pd
import numpy as np
import warnings
import importlib
import src.data_acquisition as da
import src.data_processing as dp
import src.data_visualization as dv
import src.data_interpretation as di

# Recargar los módulos para reflejar cambios recientes
importlib.reload(dp) 
importlib.reload(da)
importlib.reload(dv)
importlib.reload(di)

warnings.filterwarnings('ignore')

In [431]:
# === Listar los archivos disponibles en la carpeta de datos ===
print("="*60)
print("📂 Archivos disponibles en la carpeta")
print("="*60)
for file in da.list_data_files("data"):
    print(f"- {file}")

📂 Archivos disponibles en la carpeta
- data\delhiaqi.csv


In [432]:
# Cargar de los datos del archivo CSV en un DataFrame
df_air_quality = da.load_data_csv('data\delhiaqi.csv')

✅ Archivo 'data\delhiaqi.csv' cargado exitosamente.


In [433]:
print("=" * 60)
print("📊 Exploración inicial de los datos")
print("=" * 60)

# Mostrar filas del DataFrame
da.preview_data(df_air_quality, num_rows=5)

📊 Exploración inicial de los datos

🔍 Primeras 5 filas del DataFrame:


,date,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,2023-01-01 00:00:00,1655.58,1.66,39.41,5.90,17.88,169.29,194.64,5.83
1,2023-01-01 01:00:00,1869.20,6.82,42.16,1.99,22.17,182.84,211.08,7.66
2,2023-01-01 02:00:00,2510.07,27.72,43.87,0.02,30.04,220.25,260.68,11.40
3,2023-01-01 03:00:00,3150.94,55.43,44.55,0.85,35.76,252.90,304.12,13.55
4,2023-01-01 04:00:00,3471.37,68.84,45.24,5.45,39.10,266.36,322.80,14.19


In [434]:
# Mostrar dimensiones del DataFrame
da.get_dataframe_shape(df_air_quality)


📐 Dimensiones del DataFrame: 561 filas y 9 columnas


In [435]:
# Mostrar información detallada del DataFrame
da.get_dataframe_info(df_air_quality)


🧱 Columnas del Dataframe:
['date', 'co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3']

📊 Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    561 non-null    object 
 1   co      561 non-null    float64
 2   no      561 non-null    float64
 3   no2     561 non-null    float64
 4   o3      561 non-null    float64
 5   so2     561 non-null    float64
 6   pm2_5   561 non-null    float64
 7   pm10    561 non-null    float64
 8   nh3     561 non-null    float64
dtypes: float64(8), object(1)
memory usage: 39.6+ KB


In [436]:
# Mostrar los valores faltantes por columna
da.get_missing_values(df_air_quality)


❗ Valores faltantes por columna:
date     0
co       0
no       0
no2      0
o3       0
so2      0
pm2_5    0
pm10     0
nh3      0
dtype: int64


## 🧩 Módulo 2: Procesamiento y limpieza de datos

En esta sección se limpian y transforman los datos,
además se clasifica automáticamente cada registro según
el **Índice de Calidad del Aire (ICA)** basado en los niveles de PM2.5.


In [437]:
# SIEMPRE reasignar el limpiado
df_air_quality = dp.clean_dataframe(df_air_quality)

# ya no hace falta este parche si pusiste lo de arriba, pero no estorba:
# df_air_quality.index = df_air_quality.index.tz_localize(None)

dp.quality_report(df_air_quality)  # ahora debe dar 'h'


{'rows': 561,
 'cols': 8,
 'duplicates': 0,
 'inferred_freq': 'h',
 'missing': {'co': 0,
  'no': 0,
  'no2': 0,
  'o3': 0,
  'so2': 0,
  'pm2_5': 0,
  'pm10': 0,
  'nh3': 0}}

In [438]:
print("\n Report de Estadisticas Basicas:")
dp.descriptives(df_air_quality)


 Report de Estadisticas Basicas:


,count,mean,std,min,p05,p50,p95,max
co,561.0,3814.942210,3227.744681,654.22,1188.28,2590.18,11428.83,16876.22
no,561.0,51.181979,83.904476,0.00,0.00,13.30,236.03,425.58
no2,561.0,75.292496,42.473791,13.37,26.39,63.75,159.03,263.21
o3,561.0,30.141943,39.979405,0.00,0.00,11.80,124.45,164.51
so2,561.0,64.655936,61.073080,5.25,17.64,47.21,177.38,511.17
pm2_5,561.0,358.256364,227.359117,60.10,128.92,301.17,844.98,1310.20
pm10,561.0,420.988414,271.287026,69.08,158.34,340.90,1035.78,1499.27
nh3,561.0,26.425062,36.563094,0.63,4.31,14.82,102.34,267.51


In [ ]:
df_diario = dp.resample_agg(df_air_quality)

df_diario

,co,no,no2,o3,so2,pm2_5,pm10,nh3
date,,,,,,,,
2023-01-01,5233.765,95.215,90.480,0.370,71.045,342.115,420.650,41.290
2023-01-02,8224.490,135.900,99.390,0.000,93.460,735.595,877.325,26.220
2023-01-03,2750.400,9.445,62.035,38.985,59.845,288.730,330.530,15.515
2023-01-04,2763.750,8.210,71.630,16.005,48.395,312.235,354.570,14.125
2023-01-05,3898.620,42.690,70.945,0.150,64.850,376.225,429.235,18.490
2023-01-06,3978.730,53.645,87.740,0.030,68.665,400.585,437.805,19.130
2023-01-07,2082.825,5.730,54.155,34.330,34.335,244.085,263.000,10.640
2023-01-08,2416.610,7.155,54.835,15.200,39.340,310.070,336.200,9.245
2023-01-09,4966.740,78.680,86.710,0.175,92.505,468.895,539.285,24.700
